# 🔌 Plugins (with semantic functions)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### 🦜 Let's building a plugin consisting of semantic functions.

### Now the kernel contains all plugins ever added, unless we start removing them.
Entire plugins or individual functions can be removed from the kernel anytime.

### 📋 Let's prepare the execution settings and the kernel arguments.

In [2]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 2000,
     ResultsPerPrompt = 1,
     Temperature = 0.1
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas"
};

### and import the datetime semantic plugin

In [3]:
using System.IO;

kernel.ImportPluginFromPromptDirectory(
    Path.Combine(Directory.GetCurrentDirectory(), "Plugins", "DateTimeSemanticPlugin"), 
    "DateTimeSemanticPlugin");

choose DaysTo function from the semantic plugin and invoke it

In [4]:
var daysToFunction = kernel.Plugins["DateTimeSemanticPlugin"]["DaysTo"];

var response = await kernel.InvokeAsync(daysToFunction, kernelArguments);

display(response.GetValue<string>())

Since today is March 31, 2023, there are 269 days until Christmas on December 25, 2023.

### In this example, the plugin we've added `DateTimePlugin` is used in the templated prompt. The native function GetCurrentDate is invoked to enrich the prompt context with current date.

In [5]:
using System.ComponentModel;
public class DateTimePlugin
{
    [KernelFunction]
    [Description("Retrieves the current date")]
    public string GetCurrentDate() => DateTime.Now.ToShortDateString();
}

### Import the DateTimePlugin (consisting of native functions)

In [6]:
kernel.ImportPluginFromType<DateTimePlugin>("date");

### Create a new semantic function which is calling the native function in its prompt, then invoke the semantic function

In [7]:

var promptTemplate = "Considering that today date is {{date.GetCurrentDate}}, how many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

If today's date is April 29, 2024, we need to calculate the number of days until December 25, 2024.

From April 29 to April 30 is 1 day.
The month of May has 31 days.
June has 30 days.
July has 31 days.
August has 31 days.
September has 30 days.
October has 31 days.
November has 30 days.
And December has 25 days (up to Christmas).

Adding these together:

1 (April 29 to April 30) + 31 (May) + 30 (June) + 31 (July) + 31 (August) + 30 (September) + 31 (October) + 30 (November) + 25 (December up to Christmas) = 240 days

So, there are 240 days until Christmas from April 29, 2024.

### 👀 Let's check the rendered prompt.

In [8]:
#!import Helpers/PrintHelper.cs

await PrintHelper.PrintRenderedPromptAsync(kernel, promptTemplate, kernelArguments);

Considering that today date is 4/29/2024, how many days until Christmas?


### 🚰 For a better user experience we can may want to use with streaming 🚰.

In [9]:
await foreach (var token in kernel.InvokePromptStreamingAsync(promptTemplate, kernelArguments))
{
    Console.Write(token);
}

If today is April 29, 2024, we need to calculate the number of days until December 25, 2024.

From April 29 to April 30 is 1 day.
The month of May has 31 days.
June has 30 days.
July has 31 days.
August has 31 days.
September has 30 days.
October has 31 days.
November has 30 days.
And December has 25 days (up to Christmas).

Adding these together:

1 (April 29 to April 30) + 31 (May) + 30 (June) + 31 (July) + 31 (August) + 30 (September) + 31 (October) + 30 (November) + 25 (December up to Christmas) = 240 days.

So, there are 240 days until Christmas from April 29, 2024.

### There are many ways to build plugins in SK

### You can build them and then add them to the kernel

In [10]:
// kernel.CreatePluginFromFunctions()

// kernel.CreatePluginFromObject()

// kernel.CreatePluginFromPromptDirectory()

// kernel.CreatePluginFromType<>()


// kernel.Plugins.Add(plugin);

### Or you can add them directly (import)...

In [11]:
// kernel.ImportPluginFromPromptDirectory();           // semantic

// kernel.ImportPluginFromFunctions()                  // native

// kernel.ImportPluginFromPromptDirectory()            // semantic

// kernel.ImportPluginFromObject()                     // native

// kernel.ImportPluginFromType<>()                     // native or semantic from native

// kernel.ImportPluginFromOpenAIAsync                  // semantic from OpenAI plugin